In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

from clean_data import *
from exploratory_data_analysis import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

### Read the Starbucks Rewards Data
* portfolio.json
* profile.json
* transcript.json

In [2]:
portfolio = pd.read_json('data/portfolio.json', orient='records',lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### 1. Clean Portfolio data
1. Change the name of the 'id' column to 'offerid' 
2. Change the name of the 'duration' column to 'durationdays'  
4. One hot encode the 'offertype', 'channels' column  

In [3]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
portfolio = clean_portfolio()
portfolio

,offerid,difficulty,durationdays,reward,bogo,discount,informational,email,mobile,social,web
0,ae264e3637204a6fb9bb56bc8210ddfd,10,7,10,1,0,0,1,1,1,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,5,10,1,0,0,1,1,1,1
2,3f207df678b143eea3cee63160fa8bed,0,4,0,0,0,1,1,1,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,7,5,1,0,0,1,1,0,1
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,5,0,1,0,1,0,0,1
5,2298d6c36e964ae4a3e7e9706d1fb8c2,7,7,3,0,1,0,1,1,1,1
6,fafdcd668e3743c1bb461111dcafc2a4,10,10,2,0,1,0,1,1,1,1
7,5a8bc65990b245e5a138643cd4eb9837,0,3,0,0,0,1,1,1,1,0
8,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1,0,0,1,1,1,1
9,2906b810c7d4411798c6938adc9daaa5,10,7,2,0,1,0,1,1,0,1


### 2. Clean Profile data

In [5]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


#### Determine if any customer profile attributes contain missing values

In [6]:
profile.isnull().sum(axis=0) * 100 / profile.shape[0]

gender              12.794118
age                  0.000000
id                   0.000000
became_member_on     0.000000
income              12.794118
dtype: float64

#### Compute customer gender attribute distribution

In [7]:
gender_counts = profile['gender'].value_counts()
gender_counts *= 100 / gender_counts.sum()
gender_counts

M    57.227656
F    41.342327
O     1.430017
Name: gender, dtype: float64

#### Compute summary customer profile attribute statistics when customer income is not specified

In [8]:
profile[profile['income'].isnull()].describe()

,age,became_member_on,income
count,2175.0,2.175000e+03,0.0
mean,118.0,2.016804e+07,NaN
std,0.0,1.009105e+04,NaN
min,118.0,2.013080e+07,NaN
25%,118.0,2.016070e+07,NaN
50%,118.0,2.017073e+07,NaN
75%,118.0,2.017123e+07,NaN
max,118.0,2.018073e+07,NaN


#### Compute summary customer profile attribute statistics when customer income is specified

In [9]:
profile[profile['income'].notnull()].describe()

,age,became_member_on,income
count,14825.000000,1.482500e+04,14825.000000
mean,54.393524,2.016689e+07,65404.991568
std,17.383705,1.188565e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,42.000000,2.016052e+07,49000.000000
50%,55.000000,2.017080e+07,64000.000000
75%,66.000000,2.017123e+07,80000.000000
max,101.000000,2.018073e+07,120000.000000


#### Evaluate what year a customer became a rewards member

In [10]:
became_member_on = profile['became_member_on'].apply(convert_to_datetime)

start_year = became_member_on.apply(lambda elem: elem.year).value_counts()
start_year *= 100 / start_year.sum()
start_year

2017    38.052941
2018    24.694118
2016    20.741176
2015    10.764706
2014     4.064706
2013     1.682353
Name: became_member_on, dtype: float64

#### Clean the customer profile data  
1. Remove customers with missing income data  
2. Remove customer profiles where the gender attribute is missing
4. Transform the 'became_member_on' column to a datetime object  
5. One hot encode a customer's membership start year and age range  
7. Transform a customer's gender from a character to a number  

In [11]:
(profile,
 gender_integer_map) = clean_profile()

print("Number of user profiles: %d" % (profile.shape[0]))

Number of user profiles: 14613


#### Print the first five rows of the preprocessed (i.e. clean) customer profile data

In [12]:
profile.head()

,customerid,gender,income,2013,2014,2015,2016,2017,2018,"[10, 20)","[100, 110)","[20, 30)","[30, 40)","[40, 50)","[50, 60)","[60, 70)","[70, 80)","[80, 90)","[90, 100)"
0,0610b486422d4921ae7d2bf64640c50b,0,112000.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,78afa995795e4d85b5d9ceeca43f5fef,0,100000.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,1,70000.0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,389bc3fa690240e798340f5a15918d5c,1,53000.0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,2eeac8d8feae4a8cad5a6af0499a211d,1,51000.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


### 3. Clean the customer transcript data 

In [13]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


#### Compute the event type distribution

In [14]:
event_counts = transcript['event'].value_counts()
event_counts = pd.DataFrame(list(zip(event_counts.index.values, event_counts)),
                            columns=['event', 'count'])
event_counts

,event,count
0,transaction,138953
1,offer received,76277
2,offer viewed,57725
3,offer completed,33579


#### Compute the percentage of customer transaction and offer events

In [15]:
total_transactions = event_counts['count'].sum()

percentage_transactions = 100 * event_counts.iloc[0]['count'] / total_transactions
percentage_offers = 100 * event_counts.iloc[1:]['count'].sum() / total_transactions

print("Percentage of customer transaction events: %.1f %%" % (percentage_transactions))
print("Percentage of customer offer events: %.1f %%" % (percentage_offers))

Percentage of customer transaction events: 45.3 %
Percentage of customer offer events: 54.7 %


#### Clean the transaction data  
1. Change the name of the 'person' column to 'customerid'  
2. Remove customer id's that are not in the customer profile DataFrame  
3. Convert time variable units from hours to days  
4. Change the name of the 'time' column to 'timedays'  
5. Create a DataFrame that describes offers  
    - Create an offerid column
    - Parse the offer event type (i.e. 'received', 'viewed', or 'completed')
    - One hot encode customer offer events 
6. Create a DataFrame that describes customer transaction events
    - Parse customer transaction values  

In [16]:
offer_data, transaction = clean_transcript(profile)

Percentage of transactions removed: 12.31 %


In [17]:
offer_data[offer_data['customerid']=='4202b82c9a504e048d7e5f20fe687c6a']

,offerid,customerid,timedays,completed,received,viewed
5284,0b1e1539f2cc45b7b9fa7c272da2e1d7,4202b82c9a504e048d7e5f20fe687c6a,0,0,1,0
23539,0b1e1539f2cc45b7b9fa7c272da2e1d7,4202b82c9a504e048d7e5f20fe687c6a,6,0,0,1
29094,3f207df678b143eea3cee63160fa8bed,4202b82c9a504e048d7e5f20fe687c6a,7,0,1,0
35743,3f207df678b143eea3cee63160fa8bed,4202b82c9a504e048d7e5f20fe687c6a,7,0,0,1
35744,0b1e1539f2cc45b7b9fa7c272da2e1d7,4202b82c9a504e048d7e5f20fe687c6a,7,1,0,0
53857,9b98b8c7a33c4b65b9aebfe6a799e6d9,4202b82c9a504e048d7e5f20fe687c6a,14,0,1,0
63959,9b98b8c7a33c4b65b9aebfe6a799e6d9,4202b82c9a504e048d7e5f20fe687c6a,14,0,0,1
63960,9b98b8c7a33c4b65b9aebfe6a799e6d9,4202b82c9a504e048d7e5f20fe687c6a,14,1,0,0
75918,2906b810c7d4411798c6938adc9daaa5,4202b82c9a504e048d7e5f20fe687c6a,17,0,1,0
88102,2906b810c7d4411798c6938adc9daaa5,4202b82c9a504e048d7e5f20fe687c6a,17,0,0,1


#### Print the first five rows of the transformed customer offer data

In [ ]:
offer_data.tail()

#### Print the first five rows of the transformed customer transaction data

In [ ]:
transaction.head()

### Combine transaction, profile and offer data 
1. Select a customer's profile  
2. Select offer data and transactions data for a specific customer
3. Initialize DataFrames that describe when a customer receives, views, and completes an offer
4. Iterate over each offer a customer receives 
    - Initialize the time period when an offer is valid
    - Initialize a Boolean array that select customer transactions that fall within the valid offer time window
    - Initialize a Boolean array that selects a description of when a customer completes an offer (this array may not contain any True values)  
    - Initialize a Boolean array that selects a description of when a customer views an offer (this array may not contain any True values)
    - Determine whether the current offer was successful

In [ ]:
clean_data_csv_file = "./data/clean_data.csv"

if os.path.exists(clean_data_csv_file):
    clean_data = pd.read_csv(clean_data_csv_file)
else:
    clean_data = create_offeranalysis_dataset(profile,
                                              portfolio,
                                              offer_data,
                                              transaction)

    clean_data.to_csv(clean_data_csv_file, index=False)
    
clean_data = clean_data.drop(columns=['time',
                                      'customerid',
                                      'email'])

column_ordering = ['offerid', 'totalamount']
column_ordering.extend([elem for elem in clean_data.columns if elem not in column_ordering])

clean_data = clean_data[column_ordering]
clean_data.head()

In [ ]:
percent_success_display = initialize_percent_success(portfolio,
                                             clean_data)

In [ ]:
percent_success_display[['offerid', 'count', 'percentsuccess', 'bogo', 'discount', 'informational']]

In [ ]:
percent_success_display

#### Generate a plot that illustrates:
- How many customers were provided a specific offer
- Offer sucess rate (percentsuccess)

In [ ]:
f, ax = plt.subplots(figsize=(10, 4),
                     nrows=1,
                     ncols=2)

ax[0].bar(percent_success_display.index + 1, percent_success_display['count'])
ax[0].set_xticks(np.arange(0,10) + 1)
ax[0].set_xlabel('Offer #')
ax[0].set_ylabel('Count')

ax[1].plot(percent_success_display.index + 1,
           percent_success_display['percentsuccess'],
           linewidth=3)
ax[1].set_xticks(np.arange(0,10) + 1)
ax[1].set_xlabel('Offer #')
ax[1].set_ylabel('Percent Success')
plt.tight_layout()